In [5]:
import os
import numpy as np
from PIL import Image
import torch
import open_clip
from tqdm.notebook import tqdm



In [6]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [ ]:
# def get_clip_embedding_from_PIL_image(image):
#     image_tensor = preprocess(image).unsqueeze(0)
#     with torch.no_grad():
#         embedding = model.encode_image(image_tensor).squeeze().numpy()
#     return embedding

In [ ]:
# def generate_embeddings(image_directory):
#     embedding_list = []
#     id_list = []

#     for image_name in os.listdir(image_directory):
#         image_path = os.path.join(image_directory, image_name)
#         item_id = os.path.splitext(image_name)[0]
#         id_list.append(int(item_id))
#         try:
#             image = Image.open(image_path).convert("RGB")
#             embedding = get_clip_embedding_from_PIL_image(image)
#             embedding_list.append(embedding)
#         except Exception as e:
#             print(f"Error processing {image_name}: {e}")
        
#     return embedding_list, id_list

In [7]:
# 生成嵌入向量
all_embeddings = []
all_labels = []
class_names = sorted([d for d in os.listdir('dataset') if os.path.isdir(f'dataset/{d}')])


In [8]:
for label_idx, class_name in enumerate(tqdm(class_names, desc='categories')):
    class_dir = f'dataset/{class_name}'
    for img_name in tqdm(os.listdir(class_dir), desc=class_name, leave=False):
        try:
            img_path = os.path.join(class_dir, img_name)
            image = Image.open(img_path).convert("RGB")
            
            # 预处理和推理
            image_tensor = preprocess(image).unsqueeze(0)
            with torch.no_grad():
                embedding = model.encode_image(image_tensor).cpu().numpy()
            
            all_embeddings.append(embedding)
            all_labels.append(label_idx)
        except Exception as e:
            print(f"处理 {img_path} 失败：{str(e)}")

categories:   0%|          | 0/12 [00:00<?, ?it/s]

01:   0%|          | 0/50 [00:00<?, ?it/s]

02:   0%|          | 0/50 [00:00<?, ?it/s]

03:   0%|          | 0/50 [00:00<?, ?it/s]

04:   0%|          | 0/50 [00:00<?, ?it/s]

...

08:   0%|          | 0/50 [00:00<?, ?it/s]

09:   0%|          | 0/50 [00:00<?, ?it/s]

10:   0%|          | 0/50 [00:00<?, ?it/s]

11:   0%|          | 0/50 [00:00<?, ?it/s]

12:   0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
# 保存结果
np.save('animal_embeddings.npy', np.concatenate(all_embeddings, axis=0))
np.save('animal_labels.npy', np.array(all_labels))
print(f"生成嵌入完成！总样本数：{len(all_labels)}")

生成嵌入完成！总样本数：600
